In [3]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
import nltk
nltk.download('punkt')
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import geopandas as gpd
import urllib
import json
from dateutil.parser import parse
from datetime import datetime
import re
from collections import Counter

pd.options.display.max_columns = 120
pd.options.display.max_rows = 120
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/geoffperrin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import plotly.plotly as py
import plotly.graph_objs as go

In [5]:
data = '/Users/geoffperrin/Desktop/bloomberg_work/bogota_project/data/'

In [6]:
xl = pd.ExcelFile(data + "SDQS/SDQS 2017-enejun.xlsx")

In [7]:
xl.sheet_names

[u'DB Recibidos']

In [58]:
df = xl.parse('DB Recibidos')
#df = df[:1000]
#df.head()

In [59]:
df_1000 = df.sample(n=1000)

In [62]:
df_1000['Asunto'].iloc[0]

u'MALTRATO ANIMAL ABANDONO POR SUS DUE\xd1OS PERRITA DIO CRIA Y NO LA DEJAN ENTRAR  SUS DUE\xd1OS TIENEN SUFICIENTE ESPACIO SE EVIDENCIA QUE TU QUE TENER SUS CACHORROS FRENTE ALA AVENIDA DEBAJO DE ESCOMBROS NO HAY COMIDA NI AGUA SE HABLO CON ELLOS Y NO TIENEN LA MINIMA INTENCION DE AYUDAR '

In [63]:
es_blob = TextBlob(df_1000['Asunto'].iloc[0])

In [64]:
es_blob.translate(to='en')

TextBlob("ANIMAL ABUSE NEGLECT BY THEIR OWNERS PERRITA DIO BREEDING AND NOT LEAVE ENTER THEIR OWNERS HAVE ENOUGH SPACE IS EVIDENCE THAT YOU HAVE HER CUBS IN FRONT WING AVENUE UNDER RUBBLE NO FOOD OR WATER talk to them and have no intention of HELP")

translation should be - 'good afternoon, (the complaint) is to request the repair of the court record of my water meter, thank you'

In [70]:
en_blob = es_blob.translate(to='en')

In [71]:
en_blob.sentiment.polarity

0.0

In [72]:
df_1000['Asunto'].iloc[1]

u'INTERPONE QUERELLA'

In [73]:
es_blob = TextBlob(df_1000['Asunto'].iloc[1])
en_blob = es_blob.translate(to='en')
en_blob

TextBlob("INTERPONE WHAT YOU WANT")

In [74]:
en_blob.sentiment.polarity

0.0

In [75]:
es_blob.sentiment.polarity

0.0

In [76]:
### Translate all complaints into english and save to df

In [77]:
df_1000['Asunto_EN'] = ''
for i in range(len(df_1000)):
    es_blob = TextBlob(df['Asunto'].iloc[i])
    en_blob = es_blob.translate(to='en')
    df_1000['Asunto_EN'].iloc[i] = str(en_blob)

In [78]:
### export to excel so that I can then label rows

In [79]:
df_1000.head()

,Numero petición,Sector,Entidad,Entidad Nueva,Trasladado?,Entidad definitiva,Dependencia,Tema,Subtema,Localidad de los hechos,UPZ de los hechos,Barrio de los hecho,Estrato de los hechos,Punto atención,Canal,Tipo petición,Asunto,Fecha ingreso,estado,inicio terminos,cierre,dias gestión,Fecha registro,Nombre peticionario,Condición del peticionario,Localidad del ciudadano,UPZ del ciudadano,Barrio del ciudadano,Estrato del ciudadano,Tipo de Peticionario,Opciones de Petición,Entidad que recibe,Entidad que traslada,Asunto_EN
25383,865252017,AMBIENTE,SECRETARIA DE AMBIENTE,NaN,NaN,SECRETARIA DE AMBIENTE,OFICINA DE QUEJAS Y SOLUCIONES,AMBIENTE,CONTAMINACIÓN AMBIENTAL,1 - USAQUEN,9 - VERBENAL,BUENAVISTA,1.0,NaN,WEB,DERECHO DE PETICIÓN DE INTERÉS GENERAL,MALTRATO ANIMAL ABANDONO POR SUS DUEÑOS PERRIT...,2017-04-28,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-05-02,2017-05-26 20:13:40.976,19.0,2017-05-02,INGRID MARCELA VARGAS BERMUDEZ,NaN,1 - USAQUEN,9 - VERBENAL,BUENAVISTA,1.0,Natural,En nombre propio,NaN,NaN,GOOD AFTERNOON IS TO REQUEST THE REPAIR OF THE...
44109,595502017,GOBIERNO,SECRETARIA DE GOBIERNO,NaN,NaN,SECRETARIA DE GOBIERNO,ALCALDIA LOCAL DE SAN CRISTOBAL,GOBIERNO LOCAL,CONFLICTOS POR CONVIVENCIA,NaN,NaN,NaN,NaN,NaN,ESCRITO,DERECHO DE PETICIÓN DE INTERÉS GENERAL,INTERPONE QUERELLA,2017-03-24,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-03-27,2017-03-27 15:16:00.419,1.0,2017-03-27,CARLOS HUMBERTO CASTILLO RODRIGUEZ,NaN,NaN,NaN,NaN,NaN,Natural,En nombre propio,NaN,NaN,"INSTALLATION OF METER ON NOVEMBER 8, 2016, THE..."
84403,1377312017,"SEGURIDAD, CONVIVENCIA Y JUSTICIA (NUEVO)",SECRETARIA DE SEGURIDAD (NUEVA),NaN,NaN,SECRETARIA DE SEGURIDAD (NUEVA),SEGURIDAD EN LINEA,1 TRASLADO POR NO COMPETENCIA,TRASLADO POR NO COMPETENCIA,NaN,NaN,NaN,NaN,PLATAFORMA SEGURIDAD EN LINEA,E-MAIL,DERECHO DE PETICIÓN DE INTERÉS GENERAL,"""TODA LA VIA DE LA CRA 19 ENTRE LAS CALLES 134...",2017-06-22,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-06-23,2017-06-30 14:12:53.823,5.0,2017-06-23,ANÓNIMO,NaN,NaN,NaN,NaN,NaN,NaN,En nombre propio,IDU - INSTITUTO DE DESARROLLO URBANO,SECRETARIA DE SEGURIDAD (NUEVA),"COMPANY OF AQUEDUCT AND SEWER OF BOGOTA, ESP\n..."
124665,1324692017,SALUD,SUBRED SUR (NUEVA),NaN,NaN,SUBRED SUR (NUEVA),Unidad de Servicios de Salud Usme,SALUD,Historia clínica,NaN,NaN,NaN,NaN,USME,PRESENCIAL,SOLICITUD DE COPIA,SOLICITUD COPIA DE HISTORIA CLINICA,2017-06-15,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-06-16,2017-06-28 12:23:15.139,7.0,2017-06-16,VICENTE ALARCON CANELO,ADULTO MAYOR,NaN,NaN,NaN,2.0,Natural,En nombre propio,NaN,NaN,PLEASE REPLACE THE GRILLE OF ABSNIDERO THAT IS...
134517,1306662017,HABITAT,UAESP - UNIDAD DE SERVICIOS PUBLICOS,NaN,NaN,UAESP - UNIDAD DE SERVICIOS PUBLICOS,SUBDIRECCION ADMINISTRATIVA Y FINANCIERA,SERVICIOS PUBLICOS,GESTION DEL SERVICIO DE ALUMBRDAO PUBLICO - SU...,NaN,NaN,NaN,3.0,AVENIDA CARACAS NO. 53 - 80 PRIMER PISO,ESCRITO,DERECHO DE PETICIÓN DE INTERÉS PARTICULAR,ALUMBRADO PUBLICO. EL INTENDENTE HERBER ALONSO...,2017-06-14,NaN,NaT,NaT,NaN,2017-06-15,POLICIA METROPOLITANA,NaN,NaN,NaN,NaN,NaN,Natural,En nombre propio,NaN,NaN,I REFER TO YOUR COMMUNICATION S-2016-245954 WH...


In [80]:
df_1000.to_csv(data + 'random_quejas_English.csv', encoding='utf-8')

### sentiment / naive bayes help gathered from here:
https://www.dataquest.io/blog/naive-bayes-tutorial/

In [16]:
def count_text(text):
    # Split text into words based on whitespace.  Simple but effective.
    words = re.split("\s+", text)
    # Count up the occurence of each word.
    return Counter(words)

In [17]:
def length_text(text):
    # Split text into words based on whitespace.  Simple but effective.
    words = re.split("\s+", text)
    # Count up the occurence of each word.
    return len(words)

In [18]:
quejas_length = []
quejas_sentiment_polarity_es = []
quejas_sentiment_polarity_en = []
#text_ is how each point should be labeled - ideally I'd want to use category or something...
text_ = []
for i in df['Asunto']:
    es_blob = TextBlob(i)
    en_blob = es_blob.translate(to='en')
    print en_blob.sentiment.polarity
    print length_text(i)
    quejas_sentiment_polarity_en.append(en_blob.sentiment.polarity)
    quejas_sentiment_polarity_es.append(es_blob.sentiment.polarity)
    quejas_length.append(length_text(i))
    text_.append(i)

0.05
19
0.118445839874
263
0.0395587367178
638
-0.2
50
0.0666666666667
43
-0.0546743697479
424
0.362083333333
98
0.295238095238
102
-0.09375
36
0.0
22
0.261111111111
36
0.0
33
-0.222222222222
42
0.0
5
0.05
63
0.0555555555556
84
0.401666666667
69
0.0
41
-0.05
35
0.191666666667
25
0.14375
119
-0.2
17
-0.3
8
0.0
50
0.0
15
0.100595238095
386
0.0
28
0.111111111111
123
0.306666666667
171
0.2
40
-0.0303636363636
192
-0.133333333333
25
0.0426315789474
275
0.233333333333
90
0.3
58
-0.0121212121212
103
0.255555555556
70
0.107083333333
144
0.122222222222
45
-0.15
42
0.025
113
0.09375
95
0.0
19
-0.1
38
-0.6
8
-0.6
12
-0.6
8
0.26
19
0.285714285714
18
0.0190476190476
100
0.134040178571
474
-0.2
118
0.0
7
0.0
46
0.00833333333333
62
0.208333333333
33
-0.0789814814815
195
-0.0777777777778
90
0.11
55
0.28875
102
0.144444444444
38
0.211712361712
95
0.0
9
-0.0125
78
0.121842650104
345
0.0
5
0.35
155
-0.010101010101
15
0.129672232529
190
-0.188888888889
46
-0.25
8
-0.145833333333
11
-0.166666666667
43
-0.1

### Plotting Sentiment Score against complaint (queja) length

In [19]:
trace1 = go.Scatter(x=quejas_length, y=quejas_sentiment_polarity_es, marker={'color': 'red', 'size': "6"}, 
                    mode="markers", text=text_, name='1st Trace')
                                               
data=go.Data([trace1])
layout=go.Layout(title="Veeduría Quejas Español Sentimiento Plot", xaxis={'title':'Number of Words in Queja'}, yaxis={'title':'Sentiment Score'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='pyguide_1')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/plotly/tools.py:1336: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gperrin12/0 or inside your plot.ly account where it is named 'pyguide_1'


In [20]:
trace1 = go.Scatter(x=quejas_length, y=quejas_sentiment_polarity_en, marker={'color': 'red', 'size': "6"}, 
                    mode="markers", text=text_, name='1st Trace')
                                               
data=go.Data([trace1])
layout=go.Layout(title="Veeduría Quejas English Sentiment Plot", xaxis={'title':'Number of Words in Queja'}, yaxis={'title':'Sentiment Score'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='pyguide_1')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gperrin12/0 or inside your plot.ly account where it is named 'pyguide_1'
